<a href="https://colab.research.google.com/github/HammadN98/previsao_de_vendas/blob/main/previsao_das_vendas_lojas_Rossmann.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0. Importacoes

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
store = pd.read_csv("https://raw.githubusercontent.com/HammadN98/previsao_de_vendas/refs/heads/main/rossmann-store-sales/store.csv")
store.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [3]:
train = pd.read_csv("https://raw.githubusercontent.com/HammadN98/previsao_de_vendas/refs/heads/main/rossmann-store-sales/train.csv")
train.head()

<ipython-input-3-24519896a09f>:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv("https://raw.githubusercontent.com/HammadN98/previsao_de_vendas/refs/heads/main/rossmann-store-sales/train.csv")


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [4]:
test = pd.read_csv("https://raw.githubusercontent.com/HammadN98/previsao_de_vendas/refs/heads/main/rossmann-store-sales/test.csv")
test.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0


juntar os datasets para garantir ter todas as variaveis disponiveis

In [5]:
#Dimensoes pre-merge
store.shape, train.shape, test.shape

((1115, 10), (1017209, 9), (41088, 8))

In [6]:
train = train.merge(store, on='Store', how='left')
train.head()


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,5,2015-07-31,6064,625,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,5,2015-07-31,8314,821,1,1,0,1,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,5,2015-07-31,4822,559,1,1,0,1,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [7]:
test = test.merge(store, on='Store', how='left')
test.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,1,4,2015-09-17,1.0,1,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,3,4,2015-09-17,1.0,1,0,0,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
2,3,7,4,2015-09-17,1.0,1,0,0,a,c,24000.0,4.0,2013.0,0,NaN,NaN,NaN
3,4,8,4,2015-09-17,1.0,1,0,0,a,a,7520.0,10.0,2014.0,0,NaN,NaN,NaN
4,5,9,4,2015-09-17,1.0,1,0,0,a,c,2030.0,8.0,2000.0,0,NaN,NaN,NaN


In [8]:
#Dimensoes pOS-merge
train.shape, test.shape


((1017209, 18), (41088, 17))

## 1. Analise Exploratoria

### 1.1 Valores ausentes

In [9]:
#Valores ausentes por coluna no train
train.isnull().sum()

,0
Store,0
DayOfWeek,0
Date,0
Sales,0
Customers,0
Open,0
Promo,0
StateHoliday,0
SchoolHoliday,0
StoreType,0


In [10]:
#Valores ausentes por coluna no test
test.isnull().sum()

,0
Id,0
Store,0
DayOfWeek,0
Date,0
Open,11
Promo,0
StateHoliday,0
SchoolHoliday,0
StoreType,0
Assortment,0


A coluna usada como base para o merge ficou com 0 NaN

In [11]:
#PErcentual de valores ausentes por coluna
print(train.isnull().mean() * 100)
print(test.isnull().mean() * 100)


Store                         0.000000
DayOfWeek                     0.000000
Date                          0.000000
Sales                         0.000000
Customers                     0.000000
Open                          0.000000
Promo                         0.000000
StateHoliday                  0.000000
SchoolHoliday                 0.000000
StoreType                     0.000000
Assortment                    0.000000
CompetitionDistance           0.259730
CompetitionOpenSinceMonth    31.787764
CompetitionOpenSinceYear     31.787764
Promo2                        0.000000
Promo2SinceWeek              49.943620
Promo2SinceYear              49.943620
PromoInterval                49.943620
dtype: float64
Id                            0.000000
Store                         0.000000
DayOfWeek                     0.000000
Date                          0.000000
Open                          0.026772
Promo                         0.000000
StateHoliday                  0.000000
SchoolHoli

OS valores faltantes serao substituidos por:

CompetitionDistance -> Mediana

Open -> 0 / A loja devia estar fechada

CompetitionOpenSinceMonth, CompetitionOpenSinceYear -> 0 / indica que nao tem competidores proximos

Promo2SinceWeek, Promo2SinceYear, PromoInterval -> 0 / A loja nao participas das promocoes

In [12]:
train['CompetitionDistance']

,CompetitionDistance
0,1270.0
1,570.0
2,14130.0
3,620.0
4,29910.0
...,...
1017204,1900.0
1017205,1880.0
1017206,9260.0
1017207,870.0


In [13]:
median_value_train = train['CompetitionDistance'].median()
train['CompetitionDistance'] = train['CompetitionDistance'].fillna(median_value_train)
median_value_test = train['CompetitionDistance'].median()
train['CompetitionDistance'] = train['CompetitionDistance'].fillna(median_value_test)

train['CompetitionOpenSinceMonth'] = train['CompetitionOpenSinceMonth'].fillna(0)
test['CompetitionOpenSinceMonth'] = test['CompetitionOpenSinceMonth'].fillna(0)

train['CompetitionOpenSinceYear'] = train['CompetitionOpenSinceYear'].fillna(0)
test['CompetitionOpenSinceYear'] = test['CompetitionOpenSinceYear'].fillna(0)

train[''] = train[''].fillna(0)



In [14]:
#train['CompetitionDistance'] = train['CompetitionDistance'].fillna(train['CompetitionDistance'].median())

In [15]:
train['CompetitionDistance'].median()

2330.0